===========================================================================
# LINKEDIN POST GENERATOR WITH LANGCHAIN - GITHUB MODELS API VERSION
==========================================================================

In [ ]:
# CELL 1: INSTALL DEPENDENCIES
!pip install langchain langchain-openai langchain-community python-dotenv requests

In [ ]:
# CELL 2: IMPORT LIBRARIES
import os
import requests
from langchain.llms.base import LLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from typing import Any, List, Optional
from pydantic import Field

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [ ]:
# CELL 3: CUSTOM GITHUB MODELS LLM CLASS
class GitHubModelsLLM(LLM):
    """Custom LLM class to use GitHub Models API"""

    github_token: str = Field(...)
    model_name: str = Field(default="gpt-4o-mini")
    temperature: float = Field(default=0.7)
    max_tokens: int = Field(default=500)

    @property
    def _llm_type(self) -> str:
        return "github_models"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[Any] = None,
        **kwargs: Any,
    ) -> str:
        """Call the GitHub Models API"""

        headers = {
            "Authorization": f"Bearer {self.github_token}",
            "Content-Type": "application/json",
        }

        data = {
            "messages": [
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            "model": self.model_name,
            "temperature": self.temperature,
            "max_tokens": self.max_tokens,
        }

        try:
            response = requests.post(
                "https://models.inference.ai.azure.com/chat/completions",
                headers=headers,
                json=data
            )

            if response.status_code == 200:
                result = response.json()
                return result["choices"][0]["message"]["content"]
            else:
                error_msg = f"Error {response.status_code}: {response.text}"
                print(f"❌ API Error: {error_msg}")
                return f"Error: {error_msg}"

        except Exception as e:
            error_msg = f"Request failed: {str(e)}"
            print(f"❌ Request Error: {error_msg}")
            return f"Error: {error_msg}"

print("✅ Custom GitHub Models LLM class defined!")

✅ Custom GitHub Models LLM class defined!


https://github.com/marketplace

In [ ]:
# CELL 4: SET UP GITHUB TOKEN
GITHUB_TOKEN = 'Uses GitHub Models API instead of direct OpenAI API'

print("🔑 GitHub Token configured!")

🔑 GitHub Token configured!


In [ ]:
# CELL 5: LINKEDIN POST GENERATOR CLASS
class LinkedInPostGenerator:
    def __init__(self, github_token=None):
        """Initialize the LinkedIn Post Generator with GitHub Models API."""
        if github_token:
            self.github_token = github_token
        else:
            self.github_token = GITHUB_TOKEN

        # Initialize the GitHub Models LLM
        self.llm = GitHubModelsLLM(
            github_token=self.github_token,
            model_name="gpt-4o-mini",  # You can also use "gpt-4o" for better results
            temperature=0.7,
            max_tokens=500
        )

        # Create the prompt template with few-shot examples
        self.prompt_template = self._create_prompt_template()

        # Create the LLM chain
        self.chain = LLMChain(
            llm=self.llm,
            prompt=self.prompt_template,
            verbose=True
        )

    def _create_prompt_template(self):
        """Create a comprehensive prompt template with few-shot examples."""

        template = """
You are an expert LinkedIn content creator. Generate engaging, professional LinkedIn posts that are informative, thought-provoking, and encourage engagement. Follow these guidelines:

1. Start with a compelling hook
2. Include relevant insights or statistics when appropriate
3. Use a conversational yet professional tone
4. Include relevant hashtags (3-5)
5. End with a question or call-to-action to encourage engagement
6. Keep posts concise but impactful (150-300 words)
7. If a language other than English is specified, write the entire post in that language

Here are some examples of excellent LinkedIn posts:

Example 1:
Topic: Artificial Intelligence
Post: 🤖 The AI revolution isn't coming—it's already here.

Last week, I witnessed something remarkable: a small startup using AI to reduce their customer service response time by 80%. But here's the twist—their customer satisfaction scores actually went UP.

Why? Because AI didn't replace human connection; it enhanced it. The AI handled routine queries instantly, freeing up human agents to focus on complex problems requiring empathy and creative solutions.

The lesson? AI isn't about replacement—it's about amplification. The most successful companies aren't asking "How can AI replace our workers?" They're asking "How can AI help our workers become superheroes?"

What's your experience with AI in your workplace? Are you seeing it as a threat or an opportunity?

#ArtificialIntelligence #FutureOfWork #Innovation #Technology #Leadership

Example 2:
Topic: Remote Work
Post: 🏠 Remote work taught me something counterintuitive about productivity.

For years, I believed that being "always available" meant being productive. Then the pandemic forced us all remote, and I discovered the power of intentional boundaries.

My most productive days now include:
→ 2-hour focused work blocks with no notifications
→ Walking meetings for brainstorming sessions
→ Async communication for non-urgent matters
→ Clear start/stop times for the workday

The result? I'm doing better work in fewer hours, and my team is more engaged than ever.

Remote work isn't just about working from home—it's about working intentionally. The companies thriving in this new world aren't just adapting to remote work; they're reimagining what productivity actually means.

How has remote work changed your definition of productivity?

#RemoteWork #ProductivityTips #WorkLifeBalance #Leadership #FutureOfWork

Example 3:
Topic: Climate Change
Post: 🌍 Climate action isn't just an environmental issue—it's the business opportunity of our lifetime.

Yesterday, I met with a CEO who told me: "We didn't go green to save the planet. We went green to save our business." Their company reduced energy costs by 40% and attracted top talent who wanted to work for a purpose-driven organization.

The numbers don't lie:
• 88% of consumers want brands to help them live more sustainably
• Companies with strong ESG performance show 4.7% higher annual returns
• 75% of millennials would take a pay cut to work for a sustainable company

The climate crisis is real, but so is the opportunity. The question isn't whether your business can afford to go green—it's whether you can afford not to.

What's one sustainable practice your organization has implemented? I'd love to hear your success stories.

#ClimateAction #Sustainability #ESG #BusinessStrategy #Leadership

Now, generate a LinkedIn post for the following topic{language_instruction}:

Topic: {topic}
{language_param}

LinkedIn Post:"""

        return PromptTemplate(
            input_variables=["topic", "language_instruction", "language_param"],
            template=template
        )

    def generate_post(self, topic, language="English"):
        """Generate a LinkedIn post for the given topic and language."""

        # Prepare language-specific instructions
        if language.lower() == "english":
            language_instruction = ""
            language_param = ""
        else:
            language_instruction = f" in {language}"
            language_param = f"Language: {language}"

        # Generate the post using the chain
        try:
            result = self.chain.run(
                topic=topic,
                language_instruction=language_instruction,
                language_param=language_param
            )
            return result.strip()
        except Exception as e:
            return f"Error generating post: {str(e)}"

    def test_multiple_topics(self, topics, language="English"):
        """Test the generator with multiple topics."""
        results = {}

        print(f"🚀 Testing LinkedIn Post Generator with {len(topics)} topics")
        print(f"📝 Language: {language}")
        print("=" * 80)

        for i, topic in enumerate(topics, 1):
            print(f"\n🎯 Topic {i}: {topic}")
            print("-" * 50)

            post = self.generate_post(topic, language)
            results[topic] = post

            print(post)
            print("\n" + "=" * 80)

        return results

print("🎉 LinkedInPostGenerator class defined successfully!")

🎉 LinkedInPostGenerator class defined successfully!


In [ ]:
# CELL 6: INITIALIZE THE GENERATOR
generator = LinkedInPostGenerator()
print("✅ LinkedIn Post Generator initialized and ready to use!")

✅ LinkedIn Post Generator initialized and ready to use!


In [ ]:
# CELL 7: TEST WITH REQUIRED TOPICS
test_topics = [
    "Artificial Intelligence",
    "Remote Work",
    "Climate Change",
    "Personal Branding",
    "Data Privacy"
]

print("🎯 Testing with 5 required topics:")
for topic in test_topics:
    print(f"   • {topic}")

print("\n🔄 Generating posts... (This may take a moment)")

# Generate posts for all test topics
results = generator.test_multiple_topics(test_topics)

print(f"\n✅ Successfully generated {len(results)} LinkedIn posts!")

🎯 Testing with 5 required topics:
   • Artificial Intelligence
   • Remote Work
   • Climate Change
   • Personal Branding
   • Data Privacy

🔄 Generating posts... (This may take a moment)
🚀 Testing LinkedIn Post Generator with 5 topics
📝 Language: English

🎯 Topic 1: Artificial Intelligence
--------------------------------------------------


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert LinkedIn content creator. Generate engaging, professional LinkedIn posts that are informative, thought-provoking, and encourage engagement. Follow these guidelines:

1. Start with a compelling hook
2. Include relevant insights or statistics when appropriate
3. Use a conversational yet professional tone
4. Include relevant hashtags (3-5)
5. End with a question or call-to-action to encourage engagement
6. Keep posts concise but impactful (150-300 words)
7. If a language other than English is specified, write the entire post in that language

Here are some examples of excel

In [ ]:
# CELL 8: BONUS - MULTILINGUAL SUPPORT
print("\n🌟 BONUS: Testing Multilingual Support")
print("=" * 80)

# Test Spanish
print("🇪🇸 Testing Spanish:")
spanish_topic = "Inteligencia Artificial"
spanish_post = generator.generate_post(spanish_topic, "Spanish")
print(f"Topic: {spanish_topic}")
print(f"Post: {spanish_post}")
print("\n" + "=" * 80)

# Test French
print("🇫🇷 Testing French:")
french_topic = "Intelligence Artificielle"
french_post = generator.generate_post(french_topic, "French")
print(f"Topic: {french_topic}")
print(f"Post: {french_post}")
print("\n" + "=" * 80)

print("✅ Multilingual testing completed!")


🌟 BONUS: Testing Multilingual Support
🇪🇸 Testing Spanish:


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert LinkedIn content creator. Generate engaging, professional LinkedIn posts that are informative, thought-provoking, and encourage engagement. Follow these guidelines:

1. Start with a compelling hook
2. Include relevant insights or statistics when appropriate
3. Use a conversational yet professional tone
4. Include relevant hashtags (3-5)
5. End with a question or call-to-action to encourage engagement
6. Keep posts concise but impactful (150-300 words)
7. If a language other than English is specified, write the entire post in that language

Here are some examples of excellent LinkedIn posts:

Example 1:
Topic: Artificial Intelligence
Post: 🤖 The AI revolution isn't coming—it's already here.

Last week, I witnessed something remarkable: a small startup using AI to reduce their customer service response time by 80%. But here's the twist—their customer s

In [ ]:
# CELL 9: UTILITY FUNCTIONS FOR ADDITIONAL TESTING
def generate_single_post(topic, language="English"):
    """Generate a single LinkedIn post for any topic."""
    return generator.generate_post(topic, language)

def test_custom_topics():
    """Test with custom topics."""
    custom_topics = [
        "Cybersecurity",
        "Blockchain Technology",
        "Mental Health at Work",
        "Digital Transformation",
        "Sustainable Business"
    ]

    print("🔧 Testing with additional custom topics:")
    custom_results = generator.test_multiple_topics(custom_topics)
    return custom_results

# Example usage:
print("\n🛠️ Additional utility functions available:")
print("• generate_single_post(topic, language) - Generate a single post")
print("• test_custom_topics() - Test with 5 additional topics")
print("\n💡 Example: generate_single_post('Entrepreneurship', 'English')")


🛠️ Additional utility functions available:
• generate_single_post(topic, language) - Generate a single post
• test_custom_topics() - Test with 5 additional topics

💡 Example: generate_single_post('Entrepreneurship', 'English')


In [ ]:
# CELL 10: INTERACTIVE TESTING (OPTIONAL)
def interactive_mode():
    """Interactive mode for testing any topic."""
    print("\n🎮 Interactive Mode - Test any topic!")
    print("Enter topics to generate LinkedIn posts, or 'quit' to exit")
    print("-" * 50)

    while True:
        topic = input("Enter a topic: ").strip()
        if topic.lower() == 'quit':
            print("👋 Goodbye!")
            break

        language = input("Enter language (default: English): ").strip()
        if not language:
            language = "English"

        print(f"\n🔄 Generating post for '{topic}' in {language}...")
        print("-" * 50)

        post = generator.generate_post(topic, language)
        print(post)
        print("\n" + "=" * 80)

print("\n🎯 To run interactive mode, execute: interactive_mode()")
print("📝 To generate a single post: generate_single_post('Your Topic', 'English')")
print("🔧 To test custom topics: test_custom_topics()")


🎯 To run interactive mode, execute: interactive_mode()
📝 To generate a single post: generate_single_post('Your Topic', 'English')
🔧 To test custom topics: test_custom_topics()


In [ ]:
# CELL 11: TROUBLESHOOTING AND ALTERNATIVE MODELS
print("\n🔧 TROUBLESHOOTING GUIDE:")
print("=" * 80)
print("If you encounter issues, try these solutions:")
print("1. Verify your GitHub token has proper permissions")
print("2. Try different models: 'gpt-4o', 'gpt-4o-mini', 'gpt-3.5-turbo'")
print("3. Check your GitHub Models API access")
print("4. Reduce max_tokens if hitting limits")
print("\n💡 To change model, modify the model_name parameter in GitHubModelsLLM")
print("Example: model_name='gpt-4o' for better quality results")


🔧 TROUBLESHOOTING GUIDE:
If you encounter issues, try these solutions:
1. Verify your GitHub token has proper permissions
2. Try different models: 'gpt-4o', 'gpt-4o-mini', 'gpt-3.5-turbo'
3. Check your GitHub Models API access
4. Reduce max_tokens if hitting limits

💡 To change model, modify the model_name parameter in GitHubModelsLLM
Example: model_name='gpt-4o' for better quality results


In [ ]:
# CELL 12: FINAL SUMMARY
print("\n" + "="*80)
print("🎊 How to Run the chatbot on Colab")
print("="*80)
print("   • Created custom GitHubModelsLLM class")
print("   • Uses GitHub Models API instead of direct OpenAI API")
print("   • Properly handles GitHub personal access token")
print("   • Added error handling for API requests")
print("   • Uses gpt-4o-mini model (can be changed to gpt-4o)")
print("   • Maintains all original LangChain functionality")
print("\n✅ All requirements still met!")
print("📋 Ready for testing with GitHub Models API!")


🎊 How to Run the chatbot on Colab
   • Created custom GitHubModelsLLM class
   • Uses GitHub Models API instead of direct OpenAI API
   • Properly handles GitHub personal access token
   • Added error handling for API requests
   • Uses gpt-4o-mini model (can be changed to gpt-4o)
   • Maintains all original LangChain functionality

✅ All requirements still met!
📋 Ready for testing with GitHub Models API!
